In [7]:
import json
import numpy as np
import tensorflow.keras as keras
import tensorflow.keras.preprocessing.text as kpt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import load_model
import flask

app = flask.Flask(__name__)
# define a predict function as an endpoint 
@app.route("/predict", methods=["GET","POST"])
def predict():
    data = {"success": False}
    # get the request parameters
    params = flask.request.form["msg"]
    if (params == None):
        params = flask.request.args
    # if parameters are found, echo the msg parameter 
    if (params != None):
        # we're still going to use a Tokenizer here, but we don't need to fit it
        tokenizer = Tokenizer(num_words=3000)
        # for human-friendly printing
        labels = ['negative', 'positive']

        # read in our saved dictionary
        with open('dictionary.json', 'r') as dictionary_file:
            dictionary = json.load(dictionary_file)

        # this utility makes sure that all the words in your input
        # are registered in the dictionary
        # before trying to turn them into a matrix.
        def convert_text_to_index_array(text):
            words = kpt.text_to_word_sequence(text)
            wordIndices = []
            for word in words:
                if word in dictionary:
                    wordIndices.append(dictionary[word])
                else:
                    print("'%s' not in training corpus; ignoring." %(word))
            return wordIndices
        
        model = load_model('sentiment_model.h5')
        evalSentence = flask.request.form["msg"]
        if len(evalSentence) > 0:
            testArr = convert_text_to_index_array(evalSentence)
            input = tokenizer.sequences_to_matrix([testArr], mode='binary')
            # predict which bucket your input belongs in
            pred = model.predict(input)
            # and print it for the humans
            data["sentiment"] = labels[np.argmax(pred)]
            data["confidence"] = round(pred[0][np.argmax(pred)] * 100, 2)
            data["success"] = True
            print("%s sentiment; %.2f confidence" % (labels[np.argmax(pred)], round(pred[0][np.argmax(pred)] * 100, 2)))
        else:
            data["success"] = False
    # return a response in json format 
    return flask.jsonify(data)
# start the flask app, allow remote connections
app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/May/2020 10:30:33] "POST /predict HTTP/1.1" 200 -


positive sentiment; 97.72 confidence


127.0.0.1 - - [23/May/2020 10:30:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2020 10:31:10] "POST /predict HTTP/1.1" 200 -


'😩' not in training corpus; ignoring.
negative sentiment; 98.40 confidence
